In [1]:
#import of necessary libraries
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score, f1_score, classification_report
from sklearn.model_selection import train_test_split



import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from string import punctuation

import re

from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
data = pd.read_csv('mbti.csv')
data

,posts,type_new
0,'@0_o Please feel free to read my personal pos...,E
1,'Stewart Lee on immigration. What's wrong with...,I
2,'All of the above. I love being alone because ...,I
3,'When you're emotional but not actually sentim...,I
4,Think I've lost my comfort level in posting at...,I
...,...,...
3993,'inregardstomyself Oh not at all. I was thinki...,I
3994,'So what if a person really cares about you an...,E
3995,Season 1 Akane - INFP Shogo Makishima - ENTP K...,I
3996,"'*Sigh* That comes under nature, yo. Attract...",E


In [3]:
data_new = data.copy()

In [4]:
# train and test
y = data_new.type_new
x=data_new.drop('type_new',axis=1)
x_train, x_test, y_train, y_test =train_test_split(x,y,test_size=0.2)

In [5]:
print("shape of original dataset :", data_new.shape)
print("shape of input - training set", x_train.shape)
print("shape of output - training set", y_train.shape)
print("shape of input - testing set", x_test.shape)
print("shape of output - testing set", y_test.shape)

shape of original dataset : (3998, 2)
shape of input - training set (3198, 1)
shape of output - training set (3198,)
shape of input - testing set (800, 1)
shape of output - testing set (800,)


## Style-based

# Calculate word per sentence

In [6]:
def preposessing_mean(text):
    #remove URLs
    text = re.sub(r"https?://[^,\s]+,?", "", text)
    #replace ||| and ...
    text = text.replace('|||', '.')
    text = text.replace('...', '.')
    #sentences
    sentences = nltk.sent_tokenize(text)
    #len
    sentence_word_length = [len(sent.split()) for sent in sentences]
    #mean len
    mean_sentence_len = np.mean(sentence_word_length)
    return mean_sentence_len
#print(preposessing_mean(data['posts'][0]))

In [7]:
data_exp = x_test.copy() #calculate word per sentence

In [8]:
data_exp['word_sent'] = data_exp['posts'].apply(lambda x: preposessing_mean(x))

In [9]:
#count average number of words in sentences
sum = 0
for element in data_exp['word_sent']:
    sum += element
a_n_w = sum/len(data_exp['word_sent'])
print(a_n_w)

18.535002266514155


In [10]:
#list of word_sent
number =[]
for element in data_exp['word_sent']:
    number.append(element)

In [11]:
# predictions using word per sentence
prediction_word_per_sent=[]
for element in number:
    if element > a_n_w:
        prediction_word_per_sent.append('I')
    else:
        prediction_word_per_sent.append('E')
#print(prediction_word_per_sent)

In [12]:
check = []
for element in y_test:
    check.append(element)
#print(check)

In [13]:
#how well does it work (Calculate word per sentence)
accuracy = len([check[i] for i in range(0, len(check)) if check[i] == prediction_word_per_sent[i]]) / len(check)
print(accuracy)

0.51875


In [14]:
f1_score(check, prediction_word_per_sent, average=None)

array([0.58015267, 0.4363104 ])

In [15]:
target_names = ['E', 'I']
print(classification_report(check, prediction_word_per_sent, target_names=target_names))

              precision    recall  f1-score   support

           E       0.52      0.66      0.58       402
           I       0.52      0.37      0.44       398

    accuracy                           0.52       800
   macro avg       0.52      0.52      0.51       800
weighted avg       0.52      0.52      0.51       800



# F (POS+frequencies)

In [16]:
def prosessing(text):
    #remove URLs
    text = re.sub(r"https?://[^,\s]+,?", "", text)
    #remove |||
    text = re.sub(r"\.?\|\|\|", "", text)
    #remove number
    text = re.sub(r"\d", "", text)
    # tokenization
    tokens = nltk.word_tokenize(text)
    #punctuation removal
    clean_words = [w.strip(punctuation) for w in tokens]
    #lowercase
    clean_words = [w.lower() for w in clean_words if w != '']
    #lemma
    result = [lemmatizer.lemmatize(word) for word in clean_words]
    return result
#print(prosessing(data['posts'][0]))

In [17]:
def pos(text):
    tags = nltk.pos_tag(text)
    return tags
#print(pos(prosessing(data['posts'][0])))

In [18]:
def count_freq_noun(text):
    counter = 0
    for element in text:
        if element[1] == 'NN' or element[1] == 'NNS' or element[1] == 'NNP' or element[1] == 'NNPS':
            counter +=1
    a = len(text)
    res = counter/a
    return res

def count_freq_adj(text):
    counter = 0
    for element in text:
        if element[1] == 'JJ' or element[1] == 'JJS' or element[1] == 'JJR':
            counter +=1
    res = counter/len(text)
    return res

def count_freq_prepos(text):
    counter = 0
    for element in text:
        if element[1] == 'IN':
            counter +=1
    res = counter/len(text)
    return res

def count_freq_article(text):
    counter = 0
    for element in text:
        if element[1] == 'DT':
            counter +=1
    res = counter/len(text)
    return res

def count_freq_pronoun(text):
    counter = 0
    for element in text:
        if element[1] == 'PRP' or element[1] == 'PRP$':
            counter +=1
    res = counter/len(text)
    return res

def count_freq_verb(text):
    counter = 0
    for element in text:
        if element[1] == 'VB' or element[1] == 'VBG' or element[1] == 'VBD' or element[1] == 'VBN' or element[1] == 'VBP' or element[1] == 'VBZ':
            counter +=1
    a = len(text)
    res = counter/a
    return res

def count_freq_adverb(text):
    counter = 0
    for element in text:
        if element[1] == 'RB' or element[1] == 'RBR' or element[1] == 'RBS':
            counter +=1
    a = len(text)
    res = counter/a
    return res

def count_freq_interj(text):
    counter = 0
    for element in text:
        if element[1] == 'UH':
            counter +=1
    a = len(text)
    res = counter/a
    return res

In [19]:
def cal_f(element):
    final = count_freq_noun(element) + count_freq_adj(element) + count_freq_prepos(element) + count_freq_article(element) - count_freq_pronoun(element) - count_freq_verb(element) - count_freq_adverb(element) - count_freq_interj(element) + 100
    r = final/2
    return r

In [20]:
data_1 = x_test.copy()

In [21]:
data_1['lemmas'] = data_1['posts'].apply(lambda x: prosessing(x))
data_1['pos'] = data_1['lemmas'].apply(lambda x: pos(x))
data_1['f'] = data_1['pos'].apply(lambda x: cal_f(x))
data_1

,posts,lemmas,pos,f
518,"'I have an inattentive ADHD, not combined. Y...","[i, have, an, inattentive, adhd, not, combined...","[(i, NN), (have, VBP), (an, DT), (inattentive,...",50.047769
3072,'Jaime91 It seems to me that you are in need o...,"[jaime, it, seems, to, me, that, you, are, in,...","[(jaime, NN), (it, PRP), (seems, VBZ), (to, TO...",50.059864
1264,'Dirk Gently ESTP 9w8 > 7w8 > 3w4 sp/sx|||Ti-d...,"[dirk, gently, estp, w, w, w, sp/sxti-dom, acc...","[(dirk, NN), (gently, RB), (estp, VBZ), (w, JJ...",50.164590
3580,'I believe that she just didn't want to accept...,"[i, believe, that, she, just, did, n't, want, ...","[(i, NN), (believe, VBP), (that, IN), (she, PR...",50.079670
1407,"'intensity successes... A few days ago, I wa...","[intensity, success, a, few, day, ago, i, wa, ...","[(intensity, NN), (success, NN), (a, DT), (few...",50.083787
...,...,...,...,...
3281,'Thanks JayDubs Miss Bingley LittleDreamer ...,"[thanks, jaydubs, miss, bingley, littledreamer...","[(thanks, NNS), (jaydubs, VBP), (miss, JJ), (b...",50.067937
1892,'Like seriously how do you fall out of love wi...,"[like, seriously, how, do, you, fall, out, of,...","[(like, IN), (seriously, RB), (how, WRB), (do,...",50.047440
557,'http://www.youtube.com/watch?v=l1p_NHFd8jM&fe...,"[i, ll, write, short, story, in, the, genre, o...","[(i, NN), (ll, VBP), (write, JJ), (short, JJ),...",50.136743
2191,'http://4.bp.blogspot.com/-9QjARWd5a84/T3OXHJJ...,"[me, too, once, i, m, done, i, m, done, we, be...","[(me, PRP), (too, RB), (once, RB), (i, JJ), (m...",50.060247


In [22]:
#count average number of f
sum_f = 0
for element in data_1['f']:
    sum_f += element
f1 = sum_f/len(data_1['f'])
#print(f1)

In [23]:
# list of f mer
f_mer =[]
for element in data_1['f']:
    f_mer.append(element)
#print(f_mer)

In [24]:
# prediction f
prediction_f=[]
for element in f_mer:
    if element > f1:
        prediction_f.append('I')
    else:
        prediction_f.append('E')
#print(prediction_f)

In [25]:
#how well does it work (f)
accuracy = len([check[i] for i in range(0, len(check)) if check[i] == prediction_f[i]]) / len(check)
print(accuracy)

0.545


In [26]:
f1_score(check, prediction_f, average=None)

array([0.55825243, 0.53092784])

In [27]:
target_names = ['E', 'I']
print(classification_report(check, prediction_f, target_names=target_names))

              precision    recall  f1-score   support

           E       0.55      0.57      0.56       402
           I       0.54      0.52      0.53       398

    accuracy                           0.55       800
   macro avg       0.54      0.54      0.54       800
weighted avg       0.54      0.55      0.54       800



# Verbosity

In [28]:
def counter(text):
    li = []
    for element in text:
        if element not in li:
            li.append(element)
    return len(li)

In [29]:
data_word = x_test.copy()

In [30]:
data_word['proc'] = data_word['posts'].apply(lambda x: prosessing(x))
data_word['counter'] = data_word['proc'].apply(lambda x: counter(x))

In [31]:
count = []
for element in data_word['counter']:
    count.append(element)
    
l = []
for element in data_word['proc']:
    l.append(len(element))

In [32]:
verbosity_list = []
for element in count:
    index = count.index(element)
    n = l[index]
    d = element/n
    verbosity_list.append(d)

In [33]:
#среднее значение для verbosity
s = 0
for element in verbosity_list:
    s += element
f = s/len(verbosity_list)
print(f)

0.3980074493348924


In [34]:
pred=[]
for element in verbosity_list:
    if element > f:
        pred.append('I')
    else:
        pred.append('E')
#print(pred)

In [35]:
#how well it work
accuracy = len([check[i] for i in range(0, len(check)) if check[i] == pred[i]]) / len(check)
print(accuracy)

0.48625


In [36]:
f1_score(check, pred, average='macro')

0.48431537690617465

In [37]:
target_names = ['E', 'I']
print(classification_report(check, pred, target_names=target_names))

              precision    recall  f1-score   support

           E       0.49      0.54      0.52       402
           I       0.48      0.43      0.45       398

    accuracy                           0.49       800
   macro avg       0.49      0.49      0.48       800
weighted avg       0.49      0.49      0.48       800



In [38]:
x_train['posts'] = x_train['posts'].apply(lambda x: prosessing(x))

C:\Users\user\AppData\Local\Temp/ipykernel_8668/2280066469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['posts'] = x_train['posts'].apply(lambda x: prosessing(x))


In [39]:
x_test['posts'] = x_test['posts'].apply(lambda x: prosessing(x))

C:\Users\user\AppData\Local\Temp/ipykernel_8668/4025758674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test['posts'] = x_test['posts'].apply(lambda x: prosessing(x))


In [40]:
x_train["posts"] = x_train["posts"].apply(lambda x: ' '.join(x))
x_test["posts"] = x_test["posts"].apply(lambda x: ' '.join(x))

x_train

C:\Users\user\AppData\Local\Temp/ipykernel_8668/700683518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train["posts"] = x_train["posts"].apply(lambda x: ' '.join(x))
C:\Users\user\AppData\Local\Temp/ipykernel_8668/700683518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test["posts"] = x_test["posts"].apply(lambda x: ' '.join(x))


,posts
1839,a an enfp i can be messy but tend to catch mys...
1655,this mention stuff is annoyingwhat if piss wa ...
2433,you do n't understand what i mean i do n't say...
1807,it s mostly rough guessing and process of elim...
10,how do you react to them can you even begin to...
...,...
2127,blueyeah for sure i tend to be able to pick up...
3365,infp because both si-te and fi-ne are apparent...
3606,and long married i m a big fan of ramen follow...
2917,nfpi performance artist-this type almost alway...


## Bag of words 

In [41]:
count_vectorizer = CountVectorizer()

X_train = count_vectorizer.fit_transform(x_train["posts"])
Y_train = y_train
X_test = count_vectorizer.transform(x_test["posts"])

In [42]:
clf = LogisticRegression(random_state=0, max_iter=150).fit(X_train, Y_train)
predictions = clf.predict(X_test)

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
accuracy_score(y_test, predictions)

0.7675

In [44]:
target_names = ['E', 'I']
print(classification_report(check, predictions, target_names=target_names))

              precision    recall  f1-score   support

           E       0.78      0.76      0.77       402
           I       0.76      0.78      0.77       398

    accuracy                           0.77       800
   macro avg       0.77      0.77      0.77       800
weighted avg       0.77      0.77      0.77       800



## Tfidf

In [45]:
tfidf_vectorizer = TfidfVectorizer()


X_train = tfidf_vectorizer.fit_transform(x_train["posts"])
Y_train = y_train
X_test = tfidf_vectorizer.transform(x_test["posts"])

In [46]:
clf = LogisticRegression(random_state=0, max_iter=150).fit(X_train, Y_train)
predictions1 = clf.predict(X_test)

In [47]:
accuracy_score(y_test, predictions1)

0.79375

In [48]:
target_names = ['E', 'I']
print(classification_report(check, predictions1, target_names=target_names))

              precision    recall  f1-score   support

           E       0.82      0.76      0.79       402
           I       0.77      0.83      0.80       398

    accuracy                           0.79       800
   macro avg       0.80      0.79      0.79       800
weighted avg       0.80      0.79      0.79       800



In [64]:
pr =[]
for element in predictions:
    pr.append(element)

In [65]:
pr1 = []
for element in predictions1:
    pr1.append(element)

In [94]:
pred1 = pr
pred2 = pr1
pred3 = prediction_f
pred4 = pred
pred5 = prediction_word_per_sent
for_data = []
res = []
cof = [0.3, 0.4, 0.1, 0.1, 0.1]
counter = 0
while counter < 800:
    l = []
    l.append(pred1[counter])
    l.append(pred2[counter])
    l.append(pred3[counter])
    l.append(pred4[counter])
    l.append(pred5[counter])
    for_data.append(l)
    c =0
    r_i = 0
    r_e = 0
    cc = 0
    for element in l:
        if element == "I":
            r_i += cof[cc]
        if element == "E":
            r_e += cof[cc]
        cc +=1
    if r_i > r_e:
        res.append('I')
    else:
        res.append('E')
    counter +=1

In [68]:
table_1 = pd.DataFrame(for_data)

In [70]:
table_1

,0,1,2,3,4
0,I,I,E,E,I
1,E,E,E,E,E
2,E,E,I,I,I
3,I,I,I,E,E
4,I,I,I,E,E
...,...,...,...,...,...
795,I,I,I,E,E
796,I,I,E,E,E
797,I,I,I,I,E
798,E,E,E,I,E


In [175]:
ind = 0
fail = []
for element in check:
    if element != res[ind]:
        fail.append(ind)
    ind+=1

In [95]:
#how well does it work 
accuracy = len([check[i] for i in range(0, len(check)) if check[i] == res[i]]) / len(check)
print(accuracy)

0.805


In [96]:
target_names = ['E', 'I']
print(classification_report(check, res, target_names=target_names))

              precision    recall  f1-score   support

           E       0.80      0.81      0.81       402
           I       0.81      0.80      0.80       398

    accuracy                           0.81       800
   macro avg       0.81      0.80      0.80       800
weighted avg       0.81      0.81      0.80       800

